In [1]:
from budget_analytics.utils.fileio import FileIO
df = FileIO().read_splitwise()

In [2]:
df

,Date,Description,Category,Cost,Currency,Netty,Michael
0,2023-11-04,EasyJet,General,189.00,GBP,-94.50,94.50
1,2023-11-04,Paris hotel,Hotel,189.00,GBP,94.50,-94.50
2,2023-11-05,Hachi JP BBQ,Dining out,57.71,GBP,-28.85,28.85
3,2023-11-05,Japan centre,General,16.41,GBP,8.20,-8.20
4,2023-11-05,SeeWoo,General,24.25,GBP,-12.13,12.13
...,...,...,...,...,...,...,...
859,2025-10-30,Tokoro,General,66.90,GBP,33.45,-33.45
860,2025-10-30,Amorino,General,7.30,GBP,3.65,-3.65
861,2025-11-01,Amorino,General,7.30,GBP,7.30,-7.30
862,2025-11-01,Tian tian,General,18.28,GBP,9.14,-9.14


In [6]:
df.Category.unique()

array(['General', 'Hotel', 'Dining out', 'Groceries', 'Liquor',
       'Bus/train', 'Payment', 'TV/Phone/Internet', 'Gas/fuel',
       'Furniture', 'Electricity', 'Rent', 'Services', 'Taxi', 'Clothing',
       'Games', 'Movies', 'Bicycle', 'Sports', 'Household supplies',
       'Gifts', 'Medical expenses', 'Water', 'Cleaning',
       'Transportation - Other'], dtype=object)

In [9]:
df.loc[df.Category=="Sports"].Description.unique()


array(['Ikea food', "'- Chuanglee(1/2) +?IKEA Pot/Bathrack",
       'Ikea meatballs', 'Ikea', 'Ikea office chair', 'ikea boxes bags',
       'ikea furniture', 'Ikea Food Minus Salmon', 'First Table'],
      dtype=object)

In [ ]:
import datetime as dt
import pandas as pd

from budget_analytics.utils.fileio import CardsFileIO, FileIO
from budget_analytics.utils.recurring import expand_recurring
from budget_analytics.constants import DataSource, User, CashflowDirection, BankAccount, ExpenditureCategory

In [ ]:
expenditure_amex = CardsFileIO(user=User.MICHAEL).read_statement(BankAccount.AMEX_CASHBACK)

In [ ]:
df1 = expenditure_amex[["Description", "Category"]]
df1["Description"] = df1["Description"].str.split(r"[\s*]+").map(lambda s: " ".join(s).strip().lower())
df1.index = df1["Description"]
amex_cat_map = df1.drop(columns="Description").to_dict()["Category"]

def categorize(value: str, mapping: dict = amex_cat_map) -> str | None:
    value = value.lower()
    if value in mapping.keys():
        return mapping[value]
    else:
        for val, cat in mapping.items():
            if value in val:
                return cat
        return None

In [ ]:
expenditure_chase = CardsFileIO(user=User.MICHAEL).read_statement(BankAccount.CHASE_BA)
expenditure_chase["Category"] = expenditure_chase["Description"].map(categorize)

In [ ]:
expenditure_chase

In [ ]:
expenditure_amex.head(3)

In [ ]:
expenditure_amex.groupby(by="Date")["Amount_GBP"].sum().plot()


In [ ]:

expenditure_amex.loc[expenditure_amex.Category=="restaurant"].groupby(by="Town/City")["Amount_GBP"].sum().plot.pie()

In [ ]:
def foo(x):
    return "weekend" if x.weekday()>=5 else "weekday"

In [ ]:
df1 = expenditure_amex.groupby(by="Date")[["Amount_GBP"]].sum()

In [ ]:
df1["Weekday"]=df1.index.map(lambda x: x.weekday())

In [ ]:
df1.groupby(by="Weekday")["Amount_GBP"].mean().plot()

In [ ]:
expenditure_amex.loc["Eat Tokyo" in expenditure_amex.Description]

In [ ]:
expenditure_amex.loc[expenditure_amex.Description.str.contains("Lidl", case=False)]["Amount_GBP"].mean()

In [ ]:
expenditure_amex.loc[expenditure_amex.Description.str.contains("asda", case=False)]["Amount_GBP"].mean()

In [ ]:
df1=expenditure_amex["Description"].value_counts()

In [ ]:
df2=expenditure_amex.groupby(by="Description")["Amount_GBP"].mean().sort_values(ascending=False)

In [ ]:
pd.concat([df1,df2], axis=1).sort_values(by=["Amount_GBP"], ascending=False)

In [ ]:
expenditure_amex.loc[expenditure_amex["Description"].str.contains("MYUNGGA",case=False)]

In [ ]:
expenditure_amex["Weekday"]=expenditure_amex.Date.map(lambda x: x.weekday())
expenditure_amex.
expenditure_amex.groupby(by="Weekday")["Amount_GBP"].sum()

In [ ]:
expenditure_amex.loc[expenditure_amex.Weekday=="weekend"].groupby(by="Date")["Amount_GBP"].sum().mean()


In [ ]:
expenditure_amex.groupby("Category", sort=False).Amount_GBP.sum().plot.pie()

In [ ]:
user = User.MICHAEL
start_date = dt.date(2025, 9, 1)
end_date = dt.date(2025, 9, 30)

expenditure_amex = CardsFileIO(user=user).read_statement(BankAccount.AMEX_CASHBACK)
expenditure_amex = expenditure_amex.loc[
    (expenditure_amex.Date >= start_date) & (expenditure_amex.Date <= end_date)
]
expenditure_amex["Direction"] = CashflowDirection.OUTFLOW.value
expenditure_amex["Data_Source"] = DataSource.AMEX.value
expenditure_amex["Bank_Account"] = BankAccount.AMEX_CASHBACK.value
recurring = expand_recurring(
    FileIO(user=user).read_recurring(), start_date, end_date
)
recurring["Data_Source"] = DataSource.RECURRING.value
recurring["Bank_Account"] = None
cashflows = pd.concat([expenditure_amex[recurring.columns], recurring], axis=0).sort_values(["Date", "Amount_GBP"]).reset_index(drop=True)

In [ ]:
expenditure_amex

In [ ]:
expenditure_amex.groupby("Category", sort=False).Amount_GBP.sum().sum()

In [ ]:
from budget_analytics.constants import ExpenditureCategory
expenditure_amex.loc[expenditure_amex.Category==ExpenditureCategory.OTHER.value]

In [ ]:
recurring = expand_recurring(
    FileIO(user=user).read_recurring(), start_date, dt.date(2026,3,1)
)

In [ ]:
recurring["Cumulative_Amount"] = (recurring.Amount_GBP * recurring.Direction.map(lambda x: CashflowDirection(x).multiplier)).cumsum()

In [ ]:
recurring_net =

In [ ]:
recurring

In [ ]:
cashflows.Amount_GBP * cashflows.Direction.map(lambda x: CashflowDirection(x).multiplier)

In [ ]:
cashflows

In [ ]:
(cashflows.Amount_GBP * cashflows.Direction.map(lambda x: CashflowDirection(x).multiplier)).sum()

In [ ]:
list(FileIO(data_path="cards_michael/chase_ba").read_all_csv(skiprows=1).values())[0].reset_index()

In [ ]:
from budget_analytics.constants import Calendar
from budget_analytics.utils.calendar import get_calendar_dates

dates = get_calendar_dates(Calendar.GB, dt.date(2010,1,1), dt.date(2010,1,31))

In [ ]:
dates

In [ ]:
import holidays as hol
dt.date(2020,1,1) in hol.country_holidays(country="GB", subdiv="ENG")

In [ ]:
list(map(lambda d: d.date(), dates))

In [ ]:
dfs = FileIO(data_path="cards/amex_michael").read_all_csv()
df = pd.concat(dfs.values(), axis=0)
df["Date"] = df["Date"].map(lambda s: dt.datetime.strptime(s, "%d/%m/%Y").date())
df = df.sort_values(by=["Date"]).reset_index(drop=True)
df = df[["Date","Category","Description", "Amount","Extended Details"]]

In [ ]:
df

In [ ]:
df.sort_values("Category")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1,figsize=(10,5))
df1 = df.groupby(["Category","Date"])["Amount"].sum().reset_index("Category").pivot(columns="Category").fillna(0).cumsum()
df1.iloc[-1].sort_values(ascending=False)
plt.legend(fontsize=7)
plt.show()

In [ ]:
df1 = df.groupby(["Category","Date"])["Amount"].sum().reset_index("Category").pivot(columns="Category").fillna(0).cumsum()

In [ ]:
df1.iloc[-1].sort_values(ascending=False).index

In [ ]:
df.loc[(df.Date >= dt.date(2025, 8, 1)) & (df.Date <= dt.date(2025, 8, 31)) & (df.Category == "Entertainment-Restaurants")].Amount.sum()

In [ ]:
df.loc[(df.Date >= dt.date(2025, 8, 1)) & (df.Date <= dt.date(2025, 8, 31))].groupby("Category")["Amount"].sum()

In [ ]:
df.loc[(df.Date >= dt.date(2025, 8, 1)) & (df.Date <= dt.date(2025, 8, 31))].sort_values("Category")

In [ ]:
df.loc[(df.Date >= dt.date(2025, 8, 1)) & (df.Date <= dt.date(2025, 8, 31)) & ~df["Extended Details"].isna()]

In [ ]:
df = dfs["activity_month_20250908"]

In [ ]:
df